In [2]:
!pip install numpy==1.26.4

In [4]:
!pip install transformers datasets scikit-learn torch -q

Fatal Python error: init_import_site: Failed to import the site module
Python runtime state: initialized
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1147, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 690, in _load_unlocked
  File "<frozen importlib._bootstrap>", line 980, in exec_module
  File "<frozen site>", line 642, in <module>
  File "<frozen site>", line 629, in main
  File "<frozen site>", line 410, in addsitepackages
  File "<frozen site>", line 242, in addsitedir
  File "<frozen site>", line 201, in addpackage
  File "<string>", line 1, in <module>
  File "/usr/local/lib/python3.11/dist-packages/_numba_cuda_redirector.py", line 2, in <module>
    import importlib.abc
  File "/usr/lib/python3.11/importlib/abc.py", line 19, in <module>
    from .resources.abc import ResourceReader, Traversable, TraversableResources
  File "/usr/lib/python3.11/import

In [5]:
!pip install -U transformers

In [6]:
import transformers
print(transformers.__version__)

4.52.3


In [7]:
import pandas as pd
import numpy as np
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report
from datasets import Dataset

In [8]:
from google.colab import files
uploaded = files.upload()

Saving Fake.csv to Fake (1).csv
Saving True.csv to True (1).csv


In [9]:
# Load the True and Fake news datasets
true_df = pd.read_csv('True.csv')
fake_df = pd.read_csv('Fake.csv')

# Label data: 0 for Real, 1 for Fake
true_df['label'] = 0
fake_df['label'] = 1

# Combine and shuffle
data = pd.concat([true_df, fake_df]).reset_index(drop=True)
data = data[['text', 'label']]  # Keep only relevant columns
data = data.sample(frac=1).reset_index(drop=True)


In [10]:
# 90% train, 10% test split
train_size = int(0.9 * len(data))
train_df = data[:train_size]
test_df = data[train_size:]

In [11]:
# Load tokenizer and apply to datasets
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Convert to PyTorch format
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/40408 [00:00<?, ? examples/s]

Map:   0%|          | 0/4490 [00:00<?, ? examples/s]

In [12]:
# Load pre-trained DistilBERT model with classification head (2 labels)
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
train_df = train_df.sample(frac=0.05)  # Smaller dataset
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,                # Only 1 epoch for testing
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    logging_dir='./logs',
    logging_steps=10,
    report_to="none"
)

In [ ]:

from sklearn.model_selection import train_test_split

# Proper 90-10 stratified split
train_df, test_df = train_test_split(
    data,
    test_size=0.1,
    stratify=data['label'],
    random_state=42
)


In [14]:
# Define accuracy metric for evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    return {"accuracy": (preds == labels).mean()}

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


In [15]:
# Start training
trainer.train()

Step,Training Loss
10,0.608400
20,0.180700
30,0.016600
40,0.035000
50,0.033700
60,0.016700
70,0.047400
80,0.001800
90,0.039700
100,0.001400


TrainOutput(global_step=2526, training_loss=0.006793870513552738, metrics={'train_runtime': 2107.2555, 'train_samples_per_second': 19.176, 'train_steps_per_second': 1.199, 'total_flos': 5352742644891648.0, 'train_loss': 0.006793870513552738, 'epoch': 1.0})

In [16]:
# Evaluate performance on test data
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)

# Print classification report
print(classification_report(test_dataset['label'], preds, target_names=["Real", "Fake"]))

# Save model and tokenizer for later use
model.save_pretrained("bert_fake_news_model")
tokenizer.save_pretrained("bert_fake_news_model")

              precision    recall  f1-score   support

        Real       1.00      1.00      1.00      2104
        Fake       1.00      1.00      1.00      2386

    accuracy                           1.00      4490
   macro avg       1.00      1.00      1.00      4490
weighted avg       1.00      1.00      1.00      4490



('C:/Users/Saranya/OneDrive/Documents/GBC/Deep Learning1/bert_fake_news_model/tokenizer_config.json',
 'C:/Users/Saranya/OneDrive/Documents/GBC/Deep Learning1/bert_fake_news_model/special_tokens_map.json',
 'C:/Users/Saranya/OneDrive/Documents/GBC/Deep Learning1/bert_fake_news_model/vocab.txt',
 'C:/Users/Saranya/OneDrive/Documents/GBC/Deep Learning1/bert_fake_news_model/added_tokens.json',
 'C:/Users/Saranya/OneDrive/Documents/GBC/Deep Learning1/bert_fake_news_model/tokenizer.json')

In [17]:
def classify_news(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # move model to GPU if available

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # move inputs to same device

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)
        predicted_class = torch.argmax(probs).item()
        confidence = probs[0][predicted_class].item()
        label = "Fake" if predicted_class == 1 else "Real"
        return label, round(confidence, 4)

In [18]:
# ✅ Step 10: Try Sample Test Cases
test_cases = [
    "NASA confirms Earth will experience 15 days of darkness in November 2025.",
    "The Prime Minister announced a new education policy aiming to improve access in rural areas.",
    "Scientists say that coffee can cure cancer if consumed during solar eclipses.",
    "The World Bank is projecting steady economic growth for the next two years.",
    "Apple to replace iPhones with neural implants by 2026, leaked documents reveal."
]

for i, case in enumerate(test_cases, 1):
    label, score = classify_news(case)
    print(f"Test Case {i}:")
    print(f"📝 {case}")
    print(f"🔍 Prediction: {label} (Confidence: {score})\\n")

Test Case 1:
📝 NASA confirms Earth will experience 15 days of darkness in November 2025.
🔍 Prediction: Fake (Confidence: 0.9998)\n
Test Case 2:
📝 The Prime Minister announced a new education policy aiming to improve access in rural areas.
🔍 Prediction: Fake (Confidence: 0.9997)\n
Test Case 3:
📝 Scientists say that coffee can cure cancer if consumed during solar eclipses.
🔍 Prediction: Fake (Confidence: 0.9999)\n
Test Case 4:
📝 The World Bank is projecting steady economic growth for the next two years.
🔍 Prediction: Fake (Confidence: 0.9997)\n
Test Case 5:
📝 Apple to replace iPhones with neural implants by 2026, leaked documents reveal.
🔍 Prediction: Fake (Confidence: 0.9999)\n


In [ ]:

# ✅ Real-world samples to test generalization
real_world_cases = [
    "NASA confirms Earth will experience 15 days of darkness in November 2025.",
    "The Prime Minister announced a new education policy aiming to improve access in rural areas.",
    "Scientists say that coffee can cure cancer if consumed during solar eclipses.",
    "The World Bank is projecting steady economic growth for the next two years.",
    "Apple to replace iPhones with neural implants by 2026, leaked documents reveal."
]

for i, text in enumerate(real_world_cases):
    print(f"Sample {i+1}: {text}")
    print("Prediction:", classify_news(text))
    print()
